## Data Analysis with SQL.
* In this session we'll attempt to analyse data and answer some business questions with SQL.
* We are going to use the [database](https://github.com/lerocha/chinook-database) for this session.
* It contains information about a fictional digital music shop.

**DATABASE SCHEMA**
<img src="./images/chinook-schema.svg" alt="Chinook db schema" width="600px" height="400px">

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### Database summary

In [2]:
%%sql
SELECT name, type
    FROM sqlite_master
    WHERE type in ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Context Questions
1. **Write a query that gathers data about the invoice with an invoice_id of 5. Include the following columns**
> * The id of the track, track_id.
> * The name of the track, track_name.
> * The name of media type of the track, track_type.
> * The price that the customer paid for the track, unit_price.

2. **Add a column containing the artists name to the query from the previous question.**
> * The column should be called artist_name.
> * The column should come last in the result table.

In [12]:
%%sql
-- QN. 1
SELECT 
    i.unit_price,
    t.name AS track_name,
    mt.name AS track_type
FROM invoice_line AS i
INNER JOIN track AS t
    ON i.track_id = t.track_id
INNER JOIN media_type AS mt
    ON t.media_type_id = mt.media_type_id
LIMIT 5;

 * sqlite:///chinook.db
Done.


unit_price,track_name,track_type
0.99,Right Next Door to Hell,Protected AAC audio file
0.99,Dust N' Bones,Protected AAC audio file
0.99,Live and Let Die,Protected AAC audio file
0.99,Don't Cry (Original),Protected AAC audio file
0.99,Perfect Crime,Protected AAC audio file


In [15]:
%%sql
-- QN. 2
SELECT 
    i.unit_price,
    t.name AS track_name,
    mt.name AS track_type,
    ar.name AS artist_name
FROM invoice_line AS i
INNER JOIN track AS t
    ON i.track_id = t.track_id
INNER JOIN media_type AS mt
    ON t.media_type_id = mt.media_type_id
INNER JOIN album AS al
    ON al.album_id = t.album_id
INNER JOIN artist AS ar
    ON ar.artist_id = al.artist_id
LIMIT 5;

 * sqlite:///chinook.db
Done.


unit_price,track_name,track_type,artist_name
0.99,Right Next Door to Hell,Protected AAC audio file,Guns N' Roses
0.99,Dust N' Bones,Protected AAC audio file,Guns N' Roses
0.99,Live and Let Die,Protected AAC audio file,Guns N' Roses
0.99,Don't Cry (Original),Protected AAC audio file,Guns N' Roses
0.99,Perfect Crime,Protected AAC audio file,Guns N' Roses


### Business Questions
1. Write a query that returns information about each employee and their supervisor. (Include **employee_name, employee_title, supervisor_name & supervisor_title**).
2. Which sales employee had the most sales?
3. Get sales distribution by country.
4. Which is the most selling music genre in the store?

### QN. 1
Write a query that returns information about each employee and their supervisor. (Include **employee_name, employee_title, supervisor_name & supervisor_title**).

In [ ]:
%%sql
SELECT 
    e1.first_name || " " || e1.last_name AS employee_name,
    e1.title AS employee_title,
    e2.first_name || " " || e2.last_name AS supervisor_name,
    e2.title AS supervisor_title
FROM employee AS e1
LEFT JOIN employee AS e2 
    ON e2.employee_id=e1.reports_to
ORDER BY 1;

### QN. 2
Which sales employee had the most sales?

In [ ]:
%%sql
WITH sales_staff AS (
    SELECT em.first_name || ' ' || em.last_name AS Name, em.hire_date, i.total
    FROM employee AS em
    INNER JOIN customer AS c ON c.support_rep_id=em.employee_id
    INNER JOIN invoice AS i ON i.customer_id=c.customer_id
    WHERE em.title="Sales Support Agent"
)
SELECT Name, hire_date, SUM(total) AS total_sales
FROM sales_staff
GROUP BY 1
ORDER BY 3 DESC;

#### QN 3
Get sales distribution by country.

In [ ]:
%%sql
WITH 
    customers AS (
        SELECT 
            c.customer_id,
            i.total,
            CASE
                WHEN (
                        SELECT COUNT(*)
                        FROM customer
                        WHERE country=c.country
                ) = 1 THEN "Other"
                ELSE c.country
            END AS country
        FROM invoice AS i
        INNER JOIN customer AS c ON i.customer_id=c.customer_id
    ),
    customer_by_country AS (
        SELECT
            country,
            COUNT(DISTINCT customer_id) AS unique_customers,
            SUM(total) AS country_total_sales,
            ROUND(SUM(total)/COUNT(customer_id), 4) AS sales_per_customer
        FROM customers
        GROUP BY country
    )

SELECT *,
    ROUND(country_total_sales/unique_customers, 4) AS customer_value,
    CASE
        WHEN country="Other" THEN 1
    ELSE 0
    END AS sorting
FROM customer_by_country
ORDER BY sorting ASC, country_total_sales DESC;

#### QN 4.
Which is the most selling music genre in the store?

In [14]:
%%sql
WITH
    usa AS (
        SELECT 
            customer_id,
            city,
            state,
            country
        FROM customer
        WHERE country="USA"
    ),
    
    usa_invoices AS (
        SELECT country, i.invoice_id
        FROM usa AS c
        INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    ),
    
    usa_tracks AS (
        SELECT ui.*, t.track_id, t.genre_id
        FROM usa_invoices AS ui
        INNER JOIN invoice_line AS il ON il.invoice_id=ui.invoice_id
        INNER JOIN track AS t ON t.track_id=il.track_id
    ),
    
    usa_genres AS (
        SELECT ut.*, g.name
        FROM usa_tracks AS ut
        INNER JOIN genre AS g ON g.genre_id=ut.genre_id
    ),
    
    genres_sales AS (
        SELECT 
            name AS Genre,
            COUNT(*) AS Total_sales
        FROM usa_genres
        GROUP BY 1
        ORDER BY 2 DESC
    )
SELECT *, (CAST(Total_sales AS Float)/(SELECT SUM(Total_sales) FROM genres_sales)) * 100 AS Percentage_sales
FROM genres_sales
LIMIT 10;

 * sqlite:///chinook.db
Done.


Genre,Total_sales,Percentage_sales
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Pop,22,2.093244529019981
Latin,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


# END